# ATR Volatility Filter Strategy

In [1]:
import sys
from pathlib import Path

import matplotlib
if not hasattr(matplotlib.rcParams, '_get'):
    matplotlib.rcParams._get = matplotlib.rcParams.get

import matplotlib.pyplot as plt
import pandas as pd

project_root = Path('..').resolve()
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from data_loader import load_ohlcv_csv
from indicators import calculate_ema
from backtest import run_backtest
from qflib_metrics import qflib_metrics_from_returns
from strategy_atr_filter import average_true_range, atr_filter_signals


ImportError: attempted relative import with no known parent package

In [ ]:
df = load_ohlcv_csv('../data/OKX_BTCUSDT, 1D.csv')
df.head()


In [ ]:
df['EMA50'] = calculate_ema(df['close'], 50)
df['ATR14'] = average_true_range(df, 14)
df['ATR_median_100'] = df['ATR14'].rolling(window=100, min_periods=1).median()
df[['close', 'EMA50', 'ATR14', 'ATR_median_100']].tail()


In [ ]:
positions = atr_filter_signals(df)
positions.value_counts()


In [ ]:
bt = run_backtest(df.copy(), positions)
bt[['strategy_ret', 'equity_curve']].tail()


In [ ]:
metrics = qflib_metrics_from_returns(bt['strategy_ret'])
metrics_df = pd.DataFrame([metrics], index=['ATR_Filter'])
metrics_df


In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(df.index, df['close'], label='Close Price')
ax.plot(df.index, df['EMA50'], label='EMA 50', linestyle='--')
ax.fill_between(df.index, df['close'].min(), df['close'].max(), where=positions > 0.5, color='orange', alpha=0.1, label='Long Exposure')
ax.set_title('BTCUSDT Price with ATR Filter Signals')
ax.set_ylabel('Price (USDT)')
ax.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(bt.index, bt['equity_curve'], label='Equity Curve', color='brown')
ax.set_title('ATR Filter Strategy Equity Curve')
ax.set_ylabel('Equity (USD)')
ax.legend()
plt.show()
